## Algoritmos Genéticos
Es una técnica inspirada en la evolución biológica que utiliza selección, cruzamiento y mutación para resolver problemas complejos y encontrar soluciones óptimas. <br>
En este caso haré uso de un algoritmo genetico para determinar si un tablero de Sudoku de 3x3 generado cumple las condiciones de Sudoku

In [9]:
import random
import math

# Definir variables específicas para el Sudoku
numero_genes = 9  # La longitud del material genético de cada individuo (9 casillas del Sudoku)
tamano_poblacion = 100  # La cantidad de individuos en la población
precision = 30  # Cuántos individuos se seleccionan para reproducción (necesariamente mayor que 2)
probabilidad_mutacion = 0.4  # La probabilidad de que un individuo mute
valor_minimo_gen = 1
valor_maximo_gen = 3

<strong>crear_individuo(min, max):</strong> <br>Esta función crea un individuo aleatorio para ser utilizado en una población. Recibe como parámetros min y max, que representan los valores mínimos y máximos que los genes del individuo pueden tomar. La función utiliza una lista por comprensión para generar un cromosoma con números aleatorios dentro del rango especificado.

<strong>crear_poblacion(numero_individuos):</strong> <br> Esta función crea una población nueva de individuos. Recibe como parámetro numero_individuos, que indica la cantidad de individuos que se crearán en la población. La función utiliza una lista por comprensión y llama a la función crear_individuo para generar cada individuo de la población.

<strong>calcular_fitness(individuo):</strong> <br> Esta función calcula el fitness (aptitud) de un individuo del Sudoku. El fitness representa qué tan bueno es un individuo como solución del problema. En este caso, el fitness se calcula contando la cantidad de números repetidos en filas y columnas del Sudoku representado por el individuo. Utiliza un bucle for para recorrer las filas y columnas del Sudoku y la función set() para eliminar los elementos repetidos y obtener la cantidad de elementos únicos.

<strong>FITNESS</strong> <br>
Es una medida para evaluar el rendimiento de un individuo al problema dado <br>
Mientras menor sea el valor del fitness, mejor

In [2]:
def crear_individuo(min, max):
    # Crea un individuo del Sudoku, representado como una lista de 9 números (1 a 3)
    return [random.randint(min, max) for i in range(numero_genes)]

def crear_poblacion(numero_individuos):
    # Crea una población nueva de individuos del Sudoku
    return [crear_individuo(valor_minimo_gen, valor_maximo_gen) for i in range(numero_individuos)]

def calcular_fitness(individuo):
    # Calcula el fitness de un individuo del Sudoku, contando la cantidad de números repetidos en filas y columnas
    fitness = 0
    for i in range(3):
        fila = individuo[i * 3 : (i + 1) * 3]
        # Desde la posicion 0 hasta la 9 de 3 en 3, empezando del 0
        columna = individuo[i : 9 : 3]
        # Determinar cuantos conjuntos unicos hay
        # Cada uno determina cuantos elementos únicos hay
        # En las filas y las columnas 
        # Cuando una fila o columna es única, se cuenta
        fitness += (3 - len(set(fila))) + (3 - len(set(columna)))
    return fitness

In [12]:
print(crear_poblacion(5))

[[1, 1, 1, 1, 1, 3, 1, 3, 1], [2, 2, 2, 1, 3, 3, 1, 1, 1], [1, 1, 3, 3, 3, 3, 3, 3, 1], [1, 1, 1, 2, 3, 3, 1, 3, 1], [1, 3, 2, 2, 3, 3, 3, 1, 2]]


In [4]:
print(calcular_fitness([2, 3, 3, 1, 3, 3, 2, 3, 2]))

7


La función seleccion_y_reproduccion se encarga de realizar el proceso de selección y reproducción de una población de individuos en un algoritmo genético.


<strong>poblacion_evaluada -> </strong> Se encarga de calcular el fitness de cada individuo de la variable de entrada poblacion <br>
<br>
<strong>poblacion_ordenada -> </strong> Se encarga de ordenar la poblacion en funcion del fitness
<br>
<strong>poblacion_seleccionada -> </strong> Selecciona los mejores individuos de la poblacion ordenada con presicion se refiere a cuantos individuos tomara, en este caso 30
<br>
<strong>Reproducción -> </strong> En este paso, se mezcla el material genético de los individuos seleccionados para crear nuevos individuos y llenar la población restante. Se utiliza un bucle for que itera desde precision hasta la longitud total de la población. En cada iteración, se elige un punto de cruce aleatorio y se seleccionan dos padres al azar de la población seleccionada. Luego, se realiza un cruce genético entre los padres, donde se mezcla su material genético en los nuevos individuos.
<br>


In [13]:
def seleccion_y_reproduccion(poblacion):
    # Puntúa todos los individuos de la población y selecciona los mejores para reproducción
    poblacion_evaluada = [(indiv, calcular_fitness(indiv)) for indiv in poblacion]
    poblacion_ordenada = [indiv[0] for indiv in sorted(poblacion_evaluada, key=lambda x: x[1])]
    poblacion_seleccionada = poblacion_ordenada[precision:]
    
    # Se mezcla el material genético para crear nuevos individuos
    for i in range(precision, len(poblacion)):
        punto_cruce = random.randint(1, numero_genes - 1)  # Se elige un punto para hacer el intercambio
        padre = random.sample(poblacion_seleccionada, 2)  # Se eligen dos padres
        poblacion[i][:punto_cruce] = padre[0][:punto_cruce]  # Se mezcla el material genético de los padres en cada nuevo individuo
        poblacion[i][punto_cruce:] = padre[1][punto_cruce:]
    
    return poblacion

<strong>Mutacion -> </strong>  Se encarga e aplicar la mutacion a cada individuo de la poblacion

In [6]:
def mutacion(poblacion):
    # Se mutan los individuos al azar. Sin la mutación de nuevos genes nunca podría alcanzarse la solución.
    for i in range(precision, len(poblacion)):
        if random.random() <= probabilidad_mutacion:  # Cada individuo de la población (menos los padres) tiene una probabilidad de mutar
            punto = random.randint(0, numero_genes - 1)  # Se elige un punto al azar
            nuevo_valor = random.randint(valor_minimo_gen, valor_maximo_gen)  # y un nuevo valor para este punto
            
            # Es importante asegurarse de que el nuevo valor no sea igual al valor existente
            while nuevo_valor == poblacion[i][punto]:
                nuevo_valor = random.randint(valor_minimo_gen, valor_maximo_gen)
            
            # Se aplica la mutación
            poblacion[i][punto] = nuevo_valor
    
    return poblacion

### Funcion para imprimir el tablero

In [7]:
# Función para imprimir el Sudoku en forma de tablero
def imprimir_sudoku(individuo):
    for i in range(3):
        if i % 3 == 0 and i != 0:
            print("- - -")
        for j in range(3):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            print(individuo[i * 3 + j], end=" ")
        print()

<strong>Bucle For -> </strong>  Cada iteracion representa una generacion: <br>
1. Se crea una población inicial utilizando la función crear_poblacion, con un tamaño definido por tamano_poblacion. Esta población representa un conjunto de posibles soluciones al problema.
<br>
2. Se evalúa el fitness de cada individuo de la población inicial mediante la función calcular_fitness, y se almacena en una lista llamada poblacion_evaluada. El fitness es una medida de qué tan bueno es cada individuo en términos de su capacidad para resolver el problema.
<br>
3. La lista poblacion_evaluada se ordena en orden ascendente según el fitness de los individuos utilizando la función sorted y la clave de ordenamiento ordenar_por_segundo_elemento. Esto crea una lista llamada poblacion_ordenada en la cual los individuos con menor fitness aparecen primero.
<br>
4. Se itera a través de la lista poblacion_ordenada y se imprime cada individuo junto con su fitness. Esto muestra la población inicial ordenada según su calidad.
<br>
5. Se define el número de generaciones numero_generaciones que se utilizarán en la evolución de la población. También se inicializan variables para almacenar el mejor fitness encontrado, el número de generación actual y una bandera de salida (break_out_flag) para interrumpir el ciclo en caso de que se encuentre una solución óptima.
<br>
6. Se inicia un bucle for que representa cada generación en la evolución de la población. En cada iteración, se realiza lo siguiente:

a. Se calcula el fitness de todos los individuos en la población actual y se almacena en la lista puntuados.

b. Se actualiza el número de generación numero_generacion.

c. Se verifica si se ha encontrado una solución óptima, es decir, si se ha encontrado un individuo con fitness igual a cero. Si se encuentra una solución, se activa la bandera break_out_flag y se sale del bucle.

d. Se aplica la selección y reproducción a la población utilizando la función seleccion_y_reproduccion.

e. Se aplica la mutación a la población utilizando la función mutacion.

f. Se evalúa el fitness de cada individuo en la población evolucionada y se ordena la lista poblacion_evaluada nuevamente en orden ascendente según el fitness. Esta es la nueva poblacion_ordenada.

g. Se imprime el número de generación y los individuos de la poblacion_ordenada para mostrar el progreso de la evolución en cada generación.

7. Una vez finalizadas las generaciones, se imprime la población final ordenada según el fitness de los individuos.
<br>
8. Se itera a través de la poblacion_ordenada y se imprime cada individuo junto con su fitness. Esto muestra la población final ordenada según su calidad.
<br>
9. Se verifica si se encontró una solución óptima al verificar si el mejor_fitness es igual a cero. Si es cero, se imprime el individuo con mejor fitness y se muestra la solución a la ecuación específica que se está resolviendo. Además, se muestra en qué generación se encontró la solución. Si no se encuentra una solución óptima, se imprime un mensaje indicando que el algoritmo no pudo encontrar una solución. 


In [14]:

# Inicializar una población
poblacion = crear_poblacion(tamano_poblacion)
print("Población Inicial:")
for individuo in poblacion:
    imprimir_sudoku(individuo)
    print()

numero_generaciones = 1000
mejor_fitness = math.inf
numero_generacion = 0
break_out_flag = False

# Se evoluciona la población
for i in range(numero_generaciones):
    puntuados = [(ind, calcular_fitness(ind)) for ind in poblacion] 
    numero_generacion += 1
    for indPun in puntuados:
        if indPun[1] < mejor_fitness:
            mejor_fitness = indPun[1]
        if indPun[1] == 0:
            break_out_flag = True
            break
    
    if break_out_flag:
        break
    
    poblacion = seleccion_y_reproduccion(poblacion)
    poblacion = mutacion(poblacion)
    
    poblacion_evaluada = [(ind, calcular_fitness(ind)) for ind in poblacion]
    poblacion_ordenada = [ind[0] for ind in sorted(poblacion_evaluada, key=lambda x: x[1])]
    
    print(f"Generación número: {numero_generacion}")
    imprimir_sudoku(poblacion_ordenada[0])
    print(f"Fitness: {calcular_fitness(poblacion_ordenada[0])}\n")

print("Población final:")
imprimir_sudoku(poblacion_ordenada[0])
print(f"Fitness: {calcular_fitness(poblacion_ordenada[0])}\n")

if mejor_fitness == 0:
    print("¡Se encontró una solución para el Sudoku!")
else:
    print("El algoritmo no pudo encontrar una solución para el Sudoku.")


Población Inicial:
1 3 2 
3 2 2 
3 1 3 

1 3 1 
1 1 2 
2 3 3 

2 2 2 
1 3 1 
3 1 2 

1 3 3 
1 3 3 
3 1 1 

2 1 2 
1 2 1 
1 2 3 

1 3 3 
2 3 3 
2 1 2 

2 2 1 
2 1 1 
1 1 1 

3 2 2 
3 1 2 
3 1 1 

3 2 3 
3 3 3 
1 1 1 

2 3 3 
1 2 1 
3 3 3 

3 2 2 
2 3 1 
2 1 3 

1 1 3 
2 1 1 
2 2 2 

3 1 3 
1 3 1 
2 3 2 

3 1 2 
2 2 3 
1 3 1 

3 1 2 
1 1 1 
1 3 2 

2 2 1 
3 2 1 
3 1 2 

2 2 1 
3 3 2 
1 3 1 

1 2 1 
3 2 2 
3 1 3 

1 3 3 
1 1 1 
1 2 2 

2 3 3 
1 3 1 
2 3 2 

2 1 3 
1 1 2 
1 2 2 

3 3 2 
1 1 1 
3 1 1 

1 2 3 
3 1 2 
1 2 3 

1 2 3 
2 3 2 
3 1 3 

1 1 2 
2 1 1 
1 3 1 

3 3 1 
3 3 2 
2 3 2 

1 1 1 
2 1 2 
2 2 2 

1 2 2 
2 2 1 
1 2 2 

3 3 3 
1 2 3 
3 2 2 

3 2 3 
3 1 3 
3 2 2 

2 3 2 
3 2 3 
1 3 1 

1 2 3 
3 1 3 
3 3 1 

3 2 1 
1 2 3 
3 2 1 

2 3 3 
3 3 2 
3 1 2 

1 2 2 
1 1 1 
3 3 1 

2 2 2 
2 1 1 
1 3 2 

1 2 1 
3 1 2 
3 3 1 

2 3 3 
2 3 2 
3 2 1 

3 2 1 
1 3 2 
1 2 1 

2 1 3 
2 1 1 
3 2 1 

3 1 2 
1 2 3 
2 2 1 

2 1 2 
2 3 3 
1 2 3 

2 3 1 
1 2 2 
2 1 2 

1 3 2 
1 3 2 
1 3 2 

1 1 1 
2 2 2 